# Spark wrangle

In [1]:
# Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 11:24:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/23 11:24:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# ACQUIRE

## 1. Read the case, department, and source data into their own spark dataframes.

In [3]:
case = spark.read.csv('case.csv', sep=',', header=True, inferSchema=True)
case.show(5)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

# 2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?

In [4]:
dept = spark.read.csv('dept.csv', sep=',', header=True, inferSchema=True)
source = spark.read.csv('source.csv', sep=',', header=True, inferSchema=True)

In [5]:
source.write.csv('sources_csv', mode='overwrite')
source.write.json('sources_json', mode='overwrite')

In [6]:
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [7]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)




# 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [8]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [9]:
case = case.withColumnRenamed('SLA_due_date', 'case_due_date')

In [10]:
case = case.withColumn('case_closed', expr('case_closed == "YES"'))\
           .withColumn('case_late', expr('case_late == "YES"'))

case.select('case_closed', 'case_late').show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [11]:
fmt = 'M/d/yy H:mm'

case = case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))\
           .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))\
           .withColumn("case_due_date", to_timestamp("case_due_date", fmt))

case.select('case_opened_date', 'case_closed_date', 'case_due_date').show(5)

+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [12]:
case.filter('! case_closed')\
    .select('case_opened_date', 'case_due_date', 'num_days_late')\
    .sort(desc('num_days_late'))\
    .show(5)

+-------------------+-------------------+------------------+
|   case_opened_date|      case_due_date|     num_days_late|
+-------------------+-------------------+------------------+
|2017-01-01 13:57:00|2017-01-17 08:30:00|       348.6458333|
|2017-01-01 13:48:00|2017-01-17 08:30:00|       348.6458333|
|2017-01-02 11:26:00|2017-01-17 11:26:00|348.52356480000003|
|2017-01-03 10:01:00|2017-01-18 10:01:00|347.58256939999995|
|2017-01-05 14:39:00|2017-01-20 14:39:00|       345.3894213|
+-------------------+-------------------+------------------+
only showing top 5 rows




[Stage 11:==========================================>              (9 + 3) / 12]



In [13]:
case.filter('! case_closed')\
    .select('case_opened_date', 'case_due_date', 'SLA_days')\
    .sort(desc('SLA_days'))\
    .show(5)

+-------------------+-------------------+-----------+
|   case_opened_date|      case_due_date|   SLA_days|
+-------------------+-------------------+-----------+
|2017-09-22 08:27:00|2021-08-11 08:30:00| 1419.00191|
|2017-11-03 14:05:00|2021-09-22 14:05:00|     1419.0|
|2017-11-21 06:57:00|2021-10-07 08:30:00|1416.063981|
|2017-12-03 12:59:00|2021-10-18 08:30:00|1414.813009|
|2017-01-06 08:24:00|2020-11-12 08:30:00|1406.003623|
+-------------------+-------------------+-----------+
only showing top 5 rows




[Stage 12:====>                                                   (1 + 11) / 12]



In [14]:
case.filter('! case_closed')\
    .select('case_opened_date')\
    .sort(asc('case_opened_date'))\
    .show(5)

+-------------------+
|   case_opened_date|
+-------------------+
|2017-01-01 13:48:00|
|2017-01-01 13:57:00|
|2017-01-02 11:26:00|
|2017-01-03 10:01:00|
|2017-01-04 09:42:00|
+-------------------+
only showing top 5 rows



## 2. How many Stray Animal cases are there?

In [15]:
case.groupby('service_request_type')\
    .count()\
    .sort(desc('count'))\
    .show(truncate=False)

+---------------------------------+-----+
|service_request_type             |count|
+---------------------------------+-----+
|No Pickup                        |86855|
|Overgrown Yard/Trash             |65895|
|Bandit Signs                     |32910|
|Damaged Cart                     |30338|
|Front Or Side Yard Parking       |28794|
|Stray Animal                     |26760|
|Aggressive Animal(Non-Critical)  |24882|
|Cart Exchange Request            |22024|
|Junk Vehicle On Private Property |21473|
|Pot Hole Repair                  |20616|
|Alley-Way Maintenance            |20214|
|Lost/Stolen Cart                 |18731|
|Right Of Way/Sidewalk Obstruction|17699|
|Dead Animal - Dog                |16714|
|Cart Delivery                    |15471|
|Dead Animal - Cat                |14983|
|Animal Neglect                   |13441|
|Dead Animal - Misc               |13234|
|Trapped/Confined Animal          |11354|
|Public Nuisance(Own Animal)      |10715|
+---------------------------------

In [16]:
case.filter(case.service_request_type == lit('Stray Animal')).count()

26760

## 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [17]:
case.filter(case.dept_division == 'Field Operations')\
    .filter(case.service_request_type != 'Officer Standby')\
    .count()

113902

## 4. Convert the council_district column to a string column.

In [18]:
case = case.withColumn('council_district', col('council_district').cast('string'))

## 5. Extract the year from the case_closed_date column.

In [19]:
case.withColumn('year', year('case_closed_date'))\
    .select('service_request_type', 'case_closed_date', 'year')\
    .show(5)

+--------------------+-------------------+----+
|service_request_type|   case_closed_date|year|
+--------------------+-------------------+----+
|        Stray Animal|2018-01-01 12:29:00|2018|
|Removal Of Obstru...|2018-01-03 08:11:00|2018|
|Removal Of Obstru...|2018-01-02 07:57:00|2018|
|Front Or Side Yar...|2018-01-02 08:13:00|2018|
|Animal Cruelty(Cr...|2018-01-01 13:29:00|2018|
+--------------------+-------------------+----+
only showing top 5 rows



## 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [20]:
case.withColumn('num_hours_late', round(expr('num_days_late * 24'), 1))\
    .select('num_days_late', 'num_hours_late')\
    .show(10)

+-------------------+--------------+
|      num_days_late|num_hours_late|
+-------------------+--------------+
| -998.5087616000001|      -23964.2|
|-2.0126041669999997|         -48.3|
|       -3.022337963|         -72.5|
|       -15.01148148|        -360.3|
|0.37216435200000003|           8.9|
|       -29.74398148|        -713.9|
|       -14.70673611|        -353.0|
|       -14.70662037|        -353.0|
|       -14.70662037|        -353.0|
|       -14.70649306|        -353.0|
+-------------------+--------------+
only showing top 10 rows



In [21]:
case = case.withColumn('num_hours_late', round(expr('num_days_late * 24'), 1))

## 7. Join the case data with the source and department data.

In [22]:
case = case.join(source, 'source_id', 'left')\
           .drop(source.source_id)

case.show(2, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 num_hours_late       | -23964.2             
 source_username      | svcCRMLS             
-RECORD 1------------------------------------
 source_id            | svcCRMSS             
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03

In [23]:
case = case.join(dept, 'dept_division', 'left')\
           .drop(dept.dept_division)\
           .drop(dept.dept_name)\
           .withColumnRenamed('standardized_dept_name', 'department')\
           .withColumn('dept_subject_to_SLA', col('dept_subject_to_SLA') == 'YES')

case.show(2, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 source_id            | svcCRMLS             
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 num_hours_late       | -23964.2             
 source_username      | svcCRMLS             
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
-RECORD 1------------------------------------
 dept_division        | Storm Water          
 source_id            | svcCRMSS  

## 8. Are there any cases that do not have a request source?

In [24]:
case.filter(case.source_id == 'null').count()

0

In [25]:
case.filter(case.source_id == '').count()

0

## 9. What are the top 10 service request types in terms of number of requests?

In [26]:
case.groupBy('service_request_type')\
    .count()\
    .sort(desc('count'))\
    .show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



## 10. What are the top 10 service request types in terms of average days late?

In [27]:
case.groupBy('service_request_type')\
    .agg(round(mean('num_days_late'), 1).alias('avg_late'))\
    .sort(desc('avg_late'))\
    .show(10)

+--------------------+--------+
|service_request_type|avg_late|
+--------------------+--------+
|  Zoning: Junk Yards|   176.0|
|Labeling for Used...|   162.4|
|Record Keeping of...|   154.0|
|Signage Requied f...|   151.6|
|Storage of Used M...|   142.1|
|Zoning: Recycle Yard|   135.9|
|Donation Containe...|   131.8|
|License Requied U...|   128.8|
|Traffic Signal Gr...|   101.8|
|           Complaint|    72.9|
+--------------------+--------+
only showing top 10 rows



## 11. Does number of days late depend on department?

In [28]:
case.groupBy('department')\
    .agg(round(mean('num_days_late'), 1).alias('avg_late'))\
    .sort(desc('avg_late'))\
    .show(10)

+--------------------+--------+
|          department|avg_late|
+--------------------+--------+
|    Customer Service|    59.7|
|         Solid Waste|    -2.2|
|        Metro Health|    -4.9|
|  Parks & Recreation|    -5.3|
|Trans & Cap Impro...|   -20.6|
|DSD/Code Enforcement|   -38.4|
|Animal Care Services|  -226.5|
|        City Council|    null|
+--------------------+--------+




[Stage 52:====>                                                   (1 + 11) / 12]



## 12. How do number of days late depend on department and request type?

In [29]:
case.groupBy('department', 'service_request_type')\
    .agg(round(avg('num_days_late'), 1).alias('avg_days_late'))\
    .sort('department')\
    .show(50)

+--------------------+--------------------+-------------+
|          department|service_request_type|avg_days_late|
+--------------------+--------------------+-------------+
|Animal Care Services|Animal Bite(Non-C...|         -2.5|
|Animal Care Services|Spay/Neuter Reque...|         -7.0|
|Animal Care Services|Animal Bite(Criti...|          0.0|
|Animal Care Services|        Stray Animal|       -998.8|
|Animal Care Services|Animal Cruelty(Cr...|         -0.1|
|Animal Care Services|Aggressive Animal...|         16.7|
|Animal Care Services|Trapped/Confined ...|          0.2|
|Animal Care Services|Public Nuisance(O...|         -2.2|
|Animal Care Services|      Animal Neglect|          7.3|
|Animal Care Services|City Council Anim...|         -1.5|
|Animal Care Services|     Officer Standby|         -0.1|
|Animal Care Services|Animal Permits Re...|         22.2|
|Animal Care Services|Injured Animal(Cr...|         -0.1|
|Animal Care Services|Aggressive Animal...|          2.6|
|        City 